# Implementing Iteration

## Agenda

1. Review: Iteration
2. Details: *iterables*, *iterators*, `iter`, and `next`
3. Implementing iterators with classes
4. Implementing iterators with *generators* and `yield`

## 1. Review: Iteration

*Iteration* simply refers to the process of accessing — one by one — the items stored in some container. The order of the items, and whether or not the iteration is comprehensive, depends on the container.

In Python, we typically perform iteration using the `for` loop.

In [1]:
# e.g., iterating over a list
l = [2**x for x in range(10)]
for n in l:
    print(n)

1
2
4
8
16
32
64
128
256
512


In [2]:
# e.g., iterating over the key-value pairs in a dictionary
d = {x:2**x for x in range(10)}
for k,v in d.items():
    print(k, '=>', v)

0 => 1
1 => 2
2 => 4
3 => 8
4 => 16
5 => 32
6 => 64
7 => 128
8 => 256
9 => 512


## 2. Details: *iterables*, *iterators*, `iter`, and `next`

We can iterate over anything that is *iterable*. Intuitively, if something can be used as the source of items in a `for` loop, it is iterable.

But how does a `for` loop really work? (Review time!)

In [3]:
l = [2**x for x in range(10)]

In [10]:
it = iter(l)

In [11]:
next(it)

1

## 3. Implementing iterators with classes

In [12]:
class MyIterator:
    def __init__(self, max):
        self.max = max
        self.curr = 0
        
    # the following methods are required for iterator objects
    
    def __next__(self):
        if self.curr >= self.max:
            raise StopIteration()
        ret = self.curr
        self.curr += 1
        return ret
    
    def __iter__(self):
        return self
    # makes sense bc an iterator is something you can call next on 
    # an interable is target for a for loop 

In [5]:
it = MyIterator(10)

In [7]:
next(it)

1

In [8]:
it = MyIterator(10)
while True:
    try:
        print(next(it))
    except StopIteration:
        break

0
1
2
3
4
5
6
7
8
9


In [13]:
it = MyIterator(10) #create an iterator
for i in it: #use as target in a for loop
    print(i)
#when youve exhausted an iterator, u throw it away and it cant be used again

0
1
2
3
4
5
6
7
8
9


In [14]:
for i in it:
    print(i)
#prints nothing because within the iterator object, 
# its reached the max value hwen we ran the same code above

In [16]:
l = [2**x for x in range(10)] #created a list to l

In [17]:
for x in l:
    print(x) #traverse the list and it can keep be traversed over again

1
2
4
8
16
32
64
128
256
512


In [19]:
for x in l:
    print(x)
#its bc with a for loop, they use a diff iterator each time

1
2
4
8
16
32
64
128
256
512


this can be ran twice while the above reaches a max and can only be ran once because iterators are throwaway objects, but with a list 

something thats iterable can be made the target of a for loop, 

For a container type, we need to implement an `__iter__` method that returns an iterator.

In [51]:
class ArrayList:
    class ArrayListIterator:
        def __init__(self, lst):
            self.array_list = lst
            self.curr_idx = 0
        
        def __next__(self):
            if self.curr_idx < len(self.array_list.data):
                ret = self.array_list.data[self.curr_idx]
                self.curr_idx += 1
                return ret
            else:
                raise StopIteration() 
    #this error specifically, its the error defn to stop the for loop to break out of the containing loop
    #we dont see it when running a for loop
    def __iter__(self):
        return self
    def __init__(self):
        self.data = []
        
    def append(self, val):
        self.data.append(None)
        self.data[len(self.data)-1] = val
        
    def __iter__(self): #needs to return an iterator
        #cant use return self bc self is an arraylist, which isnt an iterator
        return ArrayList.ArrayListIterator(self)

In [52]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [53]:
l.data #built in list

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]

In [54]:
it = iter(l)

In [55]:
type(it)

__main__.ArrayList.ArrayListIterator

In [56]:
next(it)

1

In [57]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


## 4. Implementing iterators with generators and `yield`

What's a "generator"?

In [58]:
l = [2**x for x in range(10)]

In [59]:
for x in l: 
    print(x)

1
2
4
8
16
32
64
128
256
512


In [64]:
len(l)

10

In [65]:
l[5]

32

In [60]:
g = (2**x for x in range(10))
#its a generator

In [61]:
type(g)

generator

In [62]:
g

<generator object <genexpr> at 0x7fc19bdaa4f8>

In [63]:
for x in g:
    print(x)

1
2
4
8
16
32
64
128
256
512


In [66]:
len(g)

TypeError: object of type 'generator' has no len()

In [67]:
g[5]

TypeError: 'generator' object is not subscriptable

In [69]:
sum([x for x in range(1_000_000)])

499999500000

In [70]:
%timeit sum([x for x in range(1_000_000)])

52.2 ms ± 637 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%timeit sum(x for x in range(1_000_000))

45 ms ± 853 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


a generator is just like an iterator
+ its thrown away after certain number of uses
+ an iterator is defn like a class
+ a generator behaves like an iterator, but there's no class for it
+ instantiating a generator doesn’t actually compute anything until next is called on it
+ values in generators arent retrieved until next is called


In [79]:
g = (2**x + x*x + 5 for x in range(10))

In [90]:
next(g) #ran 10 times

StopIteration: 

In [93]:
def foo():
    print('hello world!')
    if False:
        yield
#yield keyword makes the foo ftn return a generator

In [94]:
g = foo()

In [97]:
next(g)
#prints hello world n then theres a stop iteration error

StopIteration: 

In [98]:
def foo():
    print('A')
    yield 
    print('B')
    yield 
    print('C')
    yield 

In [99]:
g = foo()

In [103]:
next(g) #prints A, B, C and then a stopitertion exception

StopIteration: 

with yield in the code, they're code routines n it allows the code to be called in another place
+ return this certain value, and the next time u call it, it resumes where it stopped 
+ pausing the execution of ftn to return a value, but remembers all the internal state of the code 

In [107]:
def foo(x, y):
    for z in range(x, y):
        yield 2*z

In [108]:
g = foo(5, 10)

In [109]:
next(g)
#prints 10, 12, 14, 16, 18, then stopiteration error

10

In [110]:
for i in foo(5,8):
    print(i)

10
12
14


+ a generator ftn just needs yield keyword
+ generators cna be target of for loop

In [114]:
f = foo(1,6)
g = foo(5,10)
h = foo(100,108)

In [115]:
next(f)

2

In [116]:
next(h)

200

In [117]:
#generator way to write the code from above
class ArrayList:
    def __init__(self):
        self.data = []
        
    def append(self, val):
        self.data.append(None)
        self.data[len(self.data)-1] = val
        
    def __iter__(self): #now a generator method
        for i in range(len(self.data)):
            yield self.data[i]

In [118]:
l = ArrayList()
for x in range(10):
    l.append(2**x)

In [119]:
l.data #built in list

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]

In [120]:
it = iter(l)

In [121]:
type(it) #bc iter ftn calls iter method whic returns yield keyword

generator

In [122]:
next(it)

1

In [123]:
for x in l:
    print(x)

1
2
4
8
16
32
64
128
256
512


iterator vs generator
+ iter is object oriented api
    + relies on classes 
+ gen is a keyword ftn: yield
    + relies on resuming where the previous next ran the code